# load pretrainded LM

model = BERTMLM.from_pretrained()

model.add_adapter("k")

model.activate_adapter("k")

model.train()
xxx KI

save adapter

del model

model = BERT_class

model.add_adpter("k")
model.load_adapter("xxxpath")

In [13]:
import torch
from torch.optim import AdamW
from transformers import BertTokenizer
from transformers.adapters import BertAdapterModel
import pandas as pd


In [81]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertAdapterModel.from_pretrained('bert-base-uncased')
model.add_adapter('a', set_active = True)
model.add_masked_lm_head('mlm')
# model.active_adapters = ['a']
model.to(device)

tokenizer.padding_side = "left" 
tokenizer.pad_token = '[PAD]' # to avoid an error
tokenizer.mask_token = '[MASK]'

cpu


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertAdapterModel: ['cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertAdapterModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertAdapterModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [15]:
print(model)

BertAdapterModel(
  (shared_parameters): ModuleDict()
  (bert): BertModel(
    (shared_parameters): ModuleDict()
    (invertible_adapters): ModuleDict()
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
              (prefix_tuning): PrefixTuningShim(
                (pool): PrefixTuningPool(

In [16]:
tokenizer(tokenizer.pad_token, return_tensors='pt')
# tokenizer(tokenizer.mask_token, return_tensors='pt')
# tokenizer(tokenizer.unk_token, return_tensors='pt')

{'input_ids': tensor([[101,   0, 102]]), 'token_type_ids': tensor([[0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1]])}

In [82]:
DATA_DIR = '../data/'
data_name = 'augment_result_06251547.csv'
data  = list(pd.read_csv(DATA_DIR+data_name)['generate'])[:100]


In [87]:
class MLMDateset(torch.utils.data.Dataset):
    def __init__(self, data, tokenizer):
        self.data  = data
        self.inputs = tokenizer(self.data, return_tensors="pt", padding=True)
        self.random_tensor = torch.rand(self.inputs['input_ids'].shape)
        self.masked_tensor = None
    def __len__(self):
        return len(self.data)
    def __getitem__(self, idx):
        self.create_MLM()
        return {key: torch.tensor(val[idx]).to(device) for key, val in self.inputs.items()}

    def create_MLM(self):
        self.inputs['labels'] = self.inputs['input_ids'].detach().clone()
        self.masked_tensor = (self.random_tensor < 0.15)*(self.inputs['input_ids'] != '101') * (self.inputs['input_ids'] != '102') * (self.inputs['input_ids'] != 0)
        non_zero_indices = []
        for i in range(len(self.inputs['input_ids'])):
            non_zero_indices.append(torch.flatten(self.masked_tensor[i].nonzero()).tolist())
        for i in range(len(self.inputs['input_ids'])):
            self.inputs['input_ids'][i, non_zero_indices[i]] = 103
        

In [88]:
dataset = MLMDateset(data, tokenizer)
dataloader = torch.utils.data.DataLoader(dataset, batch_size = 16, shuffle = True)
epochs = 2
optimizer = AdamW(model.parameters(), lr = 1e-5)

In [ ]:
for epoch in range(epochs):
    for i, batch in enumerate(dataloader):
        print(i)
        optimizer.zero_grad()
        outputs = model(**batch)
        loss = outputs.loss
        print(loss)
        loss.backward()
        optimizer.step()


In [19]:
model.save_adapter(save_directory='../adapters/', adapter_name = 'a', with_head = True)

# CLASSIFICATION

In [86]:
model_cls = BertAdapterModel.from_pretrained('bert-base-uncased')
model_cls.load_adapter(adapter_name_or_path='../adapters/', set_active = True)
model_cls.add_classification_head('cls')
model_cls.to(device)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertAdapterModel: ['cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertAdapterModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertAdapterModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [75]:
class CLSDataset(torch.utils.data.Dataset):
    def __init__(self, data_cls, tokenizer):
        self.data_cls  = data_cls
        self.tokenizer = tokenizer
        self.inputs = tokenizer(list(self.data_cls['example']), return_tensors="pt", padding=True)
        self.inputs['labels'] = torch.tensor(self.data_cls['label'])
    def __len__(self):
        return len(self.data_cls)
    def __getitem__(self, idx):
        return {key: torch.tensor(val[idx]).to(device) for key, val in self.inputs.items()}

In [76]:
DATA_DIR = '../data/'
data_cls_name = 'train_cls.csv'
data_cls  = pd.read_csv(DATA_DIR+data_cls_name, index_col=0).iloc[:100, :]


dataset_cls = CLSDataset(data_cls, tokenizer)
dataloader_cls = torch.utils.data.DataLoader(dataset_cls, batch_size = 16, shuffle = True)
epochs = 2
optimizer_cls = AdamW(model.parameters(), lr = 1e-5)

In [78]:
for epoch in range(epochs):
    for i, batch in enumerate(dataloader_cls):
        print(i)
        optimizer_cls.zero_grad()
        outputs = model_cls(**batch)
        loss = outputs.loss
        print(loss)
        loss.backward()
        optimizer_cls.step()

/var/folders/57/93d17htn4fvf59qnk7npkv8m0000gn/T/ipykernel_81794/4230091302.py:10: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return {key: torch.tensor(val[idx]) for key, val in self.inputs.items()}


0
tensor(0.5974, grad_fn=<NllLossBackward0>)
1
tensor(0.6184, grad_fn=<NllLossBackward0>)
2
tensor(0.6163, grad_fn=<NllLossBackward0>)
3
tensor(0.6262, grad_fn=<NllLossBackward0>)
4
tensor(0.6107, grad_fn=<NllLossBackward0>)
5
tensor(0.6345, grad_fn=<NllLossBackward0>)
6
tensor(0.6018, grad_fn=<NllLossBackward0>)
0
tensor(0.6091, grad_fn=<NllLossBackward0>)
1
tensor(0.6129, grad_fn=<NllLossBackward0>)
2
tensor(0.6190, grad_fn=<NllLossBackward0>)
3
tensor(0.5979, grad_fn=<NllLossBackward0>)
4
tensor(0.6433, grad_fn=<NllLossBackward0>)
5
tensor(0.6215, grad_fn=<NllLossBackward0>)
6
tensor(0.6017, grad_fn=<NllLossBackward0>)


In [40]:
adapter_update = False
if adapter_update == False:
    for name, para in model_cls.named_parameters():
        if 'adapter' in name:
            para.requires_grad = False

# TEST

In [34]:
inputs = tokenizer(data, return_tensors="pt", padding=True)
inputs['labels'] = inputs['input_ids']

In [35]:
random_tensor = torch.rand(inputs['input_ids'].shape)
masked_tensor = (random_tensor < 0.15)*(inputs['input_ids'] != '101') * (inputs['input_ids'] != '102') * (inputs['input_ids'] != 0)


In [36]:
non_zero_indices = []
for i in range(len(inputs['input_ids'])):
    non_zero_indices.append(torch.flatten(masked_tensor[i].nonzero()).tolist())

In [37]:
for i in range(len(inputs['input_ids'])):
    inputs['input_ids'][i, non_zero_indices[i]] = 103

In [52]:
inputs.keys()

dict_keys(['input_ids', 'token_type_ids', 'attention_mask', 'labels'])

In [9]:
output = model(**inputs)

NameError: name 'inputs' is not defined

In [45]:
output.keys()

odict_keys(['loss', 'logits'])

In [42]:
help(model.add_masked_lm_head)

Help on method add_masked_lm_head in module transformers.adapters.models.bert:

add_masked_lm_head(head_name, activation_function='gelu', overwrite_ok=False) method of transformers.adapters.models.bert.BertAdapterModel instance
    Adds a masked language modeling head on top of the model.
    
    Args:
        head_name (str): The name of the head.
        activation_function (str, optional): Activation function. Defaults to 'gelu'.
        overwrite_ok (bool, optional): Force overwrite if a head with the same name exists. Defaults to False.

